In [1]:
import numpy as np

from pathlib import Path
from rembg import remove
from PIL import Image
from skimage import feature

In [2]:
# Đọc dữ liệu ảnh. 
url = 'D:\\Projects\\Multimedia-DB\\photo-MMDB\\Banh-chung\\1.jpg'
path = Path(url)
in_img = Image.open(path)
# Xoá phần nền.
out_img = remove(in_img)
# in_img.show()
# out_img.show()

In [6]:
# Tạo nền trắng, ghép phần vật thể với phần nền mới
white_bg = Image.new("RGBA", size=in_img.size, color=(255, 255, 255))
white_bg.paste(out_img, (0, 0), mask=out_img)
# white_bg.show()
new_img = white_bg.convert("RGB")
new_img.save(path)

In [33]:
# Lấy 3 kênh màu r, g, b riêng biệt
wr = white_bg.getdata(band=0)
wg = white_bg.getdata(band=1)
wb = white_bg.getdata(band=2)

In [34]:
# Xác định biên bằng canny
gray_img = white_bg.convert("L")
matrix = np.array(gray_img.getdata()).reshape(gray_img.size)
edge_canny = feature.canny(matrix, low_threshold=10, high_threshold=30)
height, width = white_bg.size

In [35]:
# Tính số lượng pixel nằm ở khu vực nền
co = 0
for i in range(height):
    if not any(edge_canny[i]):
        co += width
        continue
    limit = 0
    for j in range(width):
        if edge_canny[i][j] == True:
            limit = j
            break
        else:
            co += 1
    for j in range(width-1, limit, -1):
        if edge_canny[i][j] == True:
            # print(i, j)
            break
        else:
            co += 1
# print(co)
# print(height * width)

In [36]:
# Xây dựng biểu đồ tần suất cho 3 kênh màu của vật thể
r_his = wr.histogram()
g_his = wg.histogram()
b_his = wb.histogram()
bg_pixel = min(co, r_his[255], g_his[255], b_his[255])
r_his[255] = r_his[255] - bg_pixel
g_his[255] = g_his[255] - bg_pixel
b_his[255] = b_his[255] - bg_pixel

In [37]:
# Hàm tính biểu đồ tần suất tích luỹ
def acc_his(arr):
    new_arr = []
    s = sum(arr)
    v = 0
    for i in range(len(arr)):
        v += arr[i]
        new_arr.append(v*100/s)
    return new_arr

In [38]:
# Xây dựng đặc trưng cho ảnh
feature = []
feature += acc_his(r_his)
feature += acc_his(g_his)
feature += acc_his(b_his)
print(len(feature))
feature = tuple(feature)

768


In [40]:
data = []
data.append(feature)

In [70]:
import mysql.connector

def getConnection():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Thang*160703",
        database="mmdb_demo"
    )
    return mydb

def insertCentroid(data):
    mydb = getConnection()
    mycursor = mydb.cursor()

    sql = "INSERT INTO centroid ("
    for i in range(256):
        sql += "r_" + str(i) + ", "
    for i in range(256):
        sql += "g_" + str(i) + ", "
    for i in range(255):
        sql += "b_" + str(i) + ", "
    sql += "b_255)"

    sql += " VALUES ("
    for i in range(767):
        sql += "%s, "
    sql += "%s)"
    val = data
    mycursor.executemany(sql, val)
    mydb.commit()
    print(mycursor.rowcount, "inserted.")
    mydb.close()

def getAllData(table: str):
    mydb = getConnection()
    mycursor = mydb.cursor()

    mycursor.execute("SELECT * FROM " + table)
    myresult = mycursor.fetchall()
    mydb.close()

    return myresult

def deleteAll(table: str):
    mydb = getConnection()
    mycursor = mydb.cursor()
    sql = "DELETE FROM " + table
    mycursor.execute(sql)
    mydb.commit()
    print(mycursor.rowcount, "record(s) deleted")


In [72]:
# insertCentroid(data=data)
result = getAllData(table="centroid")
print(result)
# deleteAll(table="centroid")

[(3, 0.0, 0.0108118, 0.0250518, 0.0466754, 0.0696175, 0.101789, 0.141081, 0.180636, 0.220455, 0.263703, 0.305368, 0.351779, 0.39951, 0.446976, 0.488377, 0.532679, 0.579355, 0.630777, 0.672442, 0.715162, 0.763947, 0.807458, 0.848595, 0.878394, 0.910038, 0.949593, 0.986248, 1.02264, 1.06035, 1.10386, 1.14236, 1.18218, 1.22305, 1.2634, 1.30243, 1.34356, 1.38602, 1.43138, 1.47858, 1.52051, 1.56798, 1.61782, 1.67161, 1.7138, 1.76892, 1.82931, 1.88125, 1.93241, 1.98225, 2.04238, 2.09986, 2.1563, 2.21853, 2.27681, 2.33825, 2.40339, 2.47617, 2.54605, 2.61909, 2.69161, 2.76598, 2.84219, 2.91682, 2.99513, 3.07846, 3.15362, 3.23669, 3.31553, 3.40097, 3.49406, 3.57396, 3.67048, 3.76515, 3.85639, 3.95844, 4.05786, 4.16413, 4.27647, 4.39434, 4.52092, 4.65462, 4.77513, 4.92227, 5.06151, 5.2287, 5.38613, 5.54593, 5.71312, 5.88426, 6.05725, 6.23551, 6.41457, 6.60048, 6.7972, 7.01739, 7.23205, 7.45698, 7.67401, 7.89157, 8.12837, 8.37704, 8.62018, 8.88256, 9.16182, 9.43871, 9.73063, 10.0289, 10.3369, 10.